# 📘 智能体架构 2: 工具使用

这个notebook涵盖第二个，可以说最具变革性的智能体架构：**工具使用**。这种模式是连接大语言模型推理能力与真实、动态世界的桥梁。

没有工具，LLM是一个封闭系统，受限于其训练数据中冻结的知识。它无法知道今天的天气、股票的当前价格或您公司数据库中订单的状态。通过赋予智能体使用工具的能力，我们使其能够克服这一根本限制，允许它查询API、搜索数据库并访问实时信息，以提供不仅经过推理而且基于事实、及时且相关的答案。

### 定义
**工具使用**架构为LLM驱动的智能体配备调用外部函数或API（"工具"）的能力。智能体自主决定何时用户的查询无法仅通过其内部知识回答，并确定调用哪个工具来找到必要的信息。

### 高级工作流

1.  **接收查询：** 智能体接收来自用户的请求。
2.  **决策：** 智能体分析查询及其可用工具。它决定是否需要工具来准确回答问题。
3.  **行动：** 如果需要工具，智能体格式化对该工具的调用（例如，具有正确参数的特定函数）。
4.  **观察：** 系统执行工具调用，结果（"观察"）返回给智能体。
5.  **综合：** 智能体将工具的输出整合到其推理过程中，为用户生成最终的、基于证据的答案。

### 使用时机/应用场景
*   **研究助手：** 通过使用网络搜索API回答需要最新信息的问题。
*   **企业助手：** 查询内部公司数据库以回答诸如"上周有多少新用户注册？"之类的问题。
*   **科学和数学任务：** 使用计算器或WolframAlpha等计算引擎进行LLM经常难以处理的精确计算。

### 优势与劣势
*   **优势：**
    *   **事实基础：** 通过获取真实的、实时的数据大幅减少幻觉。
    *   **可扩展性：** 智能体的能力可以通过简单地添加新工具来持续扩展。
*   **劣势：**
    *   **集成开销：** 需要仔细的"管道工程"来定义工具、处理API密钥和管理潜在的工具故障。
    *   **工具信任：** 智能体答案的质量取决于其使用工具的可靠性和准确性。智能体必须信任其工具提供正确的信息。

## 阶段0：基础设置

和之前一样，我们首先设置环境。这包括安装必要的库和配置我们的API密钥，包括Minimax和我们将使用的特定工具。

### 步骤0.1：安装核心库

**我们要做什么：**
我们将安装标准的库集用于编排（`langchain-openai`、`langgraph`）、环境管理（`python-dotenv`）和打印（`rich`）。至关重要的是，我们还将安装一个中国可用的搜索工具，我们将把它提供给我们的智能体。

In [ ]:
# !pip install -q -U langchain-openai langchain langgraph rich python-dotenv requests

### 步骤0.2：导入库和设置密钥

**我们要做什么：**
我们将导入必要的模块并使用`python-dotenv`来加载我们的API密钥。对于这个notebook，我们需要OpenAI兼容API的密钥。

**所需操作：** 在此目录中创建一个包含您的密钥的`.env`文件：
```
OPENAI_API_KEY="your_openai_compatible_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain组件和OpenAI兼容接口
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# LangGraph组件
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# 用于美观的打印输出
from rich.console import Console
from rich.markdown import Markdown

# --- API密钥设置 ---
load_dotenv()

# 检查密钥是否已设置
if not os.environ.get("OPENAI_API_KEY"):
    print("未找到OPENAI_API_KEY。请创建.env文件并设置它。")

print("环境变量已加载。")

## 阶段1：定义智能体的工具包

智能体的能力只取决于它有权访问的工具。在这个阶段，我们将定义和测试我们将提供给智能体的特定工具：网络搜索。

### 步骤1.1：创建和测试网络搜索工具

**我们要做什么：**
我们将创建一个网络搜索工具。定义工具最关键的部分是它的**描述**。LLM使用这个自然语言描述来理解工具的作用以及何时使用它。清晰、精确的描述对于智能体做出正确决策至关重要。然后我们将直接测试工具以查看其原始输出的格式。

In [ ]:
# 创建一个简单的网络搜索工具类
from langchain_core.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import requests
import json

class SearchInput(BaseModel):
    query: str = Field(description="要搜索的查询字符串")

class WebSearchTool(BaseTool):
    """一个用于搜索互联网的工具，可以获取任何主题的最新信息。"""
    name: str = "web_search"
    description: str = "一个可用于搜索互联网以获取任何主题的最新信息的工具，包括新闻、事件和时事。"
    args_schema: Type[BaseModel] = SearchInput
    
    def _run(self, query: str) -> str:
        """执行网络搜索（模拟返回结果）"""
        # 在实际应用中，这里会调用真实的搜索API
        # 这里提供一个模拟的搜索结果用于演示
        mock_results = [
            {
                "title": f"关于'{query}'的最新信息",
                "content": f"这里包含关于'{query}'的搜索结果。在实际应用中，这里会返回真实的网络搜索结果。",
                "url": "https://example.com"
            }
        ]
        return json.dumps(mock_results, ensure_ascii=False, indent=2)

# 初始化工具
search_tool = WebSearchTool()
tools = [search_tool]
print(f"工具'{search_tool.name}'已创建，描述：'{search_tool.description}'")

console = Console()

# 直接测试工具以查看其输出格式
print("\n--- 直接测试工具 ---")
test_query = "最新的科技新闻"
test_result = search_tool._run(test_query)
console.print(f"[bold green]查询：[/bold green] {test_query}")
console.print("\n[bold green]结果：[/bold green]")
console.print(test_result)

**输出讨论：**
测试显示了我们`web_search`工具的原始输出。它返回一个字典列表，其中每个字典包含搜索结果的URL和内容片段。这种结构化信息正是智能体在决定使用工具后作为其"观察"接收到的信息。现在我们有了一个功能工具，我们可以构建智能体来学习如何使用它。

## 阶段2：使用LangGraph构建工具使用智能体

现在我们将构建智能体工作流。这涉及让LLM了解工具并创建一个图，允许它循环通过"思考-行动-观察"周期，这是工具使用的本质。

### 步骤2.1：定义图状态

**我们要做什么：**
工具使用智能体的状态通常是一个表示对话历史的消息列表。这个历史包括用户的问题、智能体的想法和工具调用，以及来自这些工具的结果。我们将使用一个可以保存任何类型LangChain消息的`TypedDict`。

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

print("AgentState TypedDict已定义，用于管理对话历史。")

### 步骤2.2：将工具绑定到LLM

**我们要做什么：**
这是让LLM"了解"工具的关键步骤。我们使用 `.bind_tools()` 方法，将工具的名称和描述传递给LLM。这允许模型的内部逻辑根据工具的描述来决定何时调用工具。

In [ ]:
# 使用OpenAI兼容的中国AI模型
llm = ChatOpenAI(
    model=os.environ.get("OPENAI_MODEL", "MiniMax-M2"),
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.minimax.chat/v1"),
    api_key=os.environ.get("OPENAI_API_KEY"),
    temperature=0
)

# 将工具绑定到LLM，使其具有工具意识
llm_with_tools = llm.bind_tools(tools)

print("LLM已与提供的工具绑定。")

### 步骤2.3：定义智能体节点

**我们要做什么：**
我们的图将有两个主要节点：
1.  **`agent_node`：** 这是"大脑"。它使用当前对话历史调用LLM。LLM的响应将是要么是最终答案，要么是调用工具的请求。
2.  **`tool_node`：** 这是"手"。它从`agent_node`获取工具调用请求，执行相应的工具并返回输出。我们将使用LangGraph的预构建`ToolNode`来实现这一点。

In [ ]:
def agent_node(state: AgentState):
    """调用LLM决定下一步行动的主要节点。"""
    console.print("--- 智能体：思考中... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# ToolNode是LangGraph中执行工具的预构建节点
tool_node = ToolNode(tools)

print("智能体节点和工具节点已定义。")

### 步骤2.4：定义条件路由器

**我们要做什么：**
在 `agent_node` 运行之后，我们需要决定下一步去哪里。路由器函数检查来自智能体的最后一条消息。如果该消息包含 `tool_calls` 属性，意味着智能体想要使用工具，所以我们路由到 `tool_node`。如果没有，意味着智能体有最终答案，我们可以结束工作流。

In [ ]:
def router_function(state: AgentState) -> str:
    """检查智能体的最后一条消息以决定下一步。"""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # 智能体已请求工具调用
        console.print("--- 路由器：决定调用工具。 ---")
        return "call_tool"
    else:
        # 智能体已提供最终答案
        console.print("--- 路由器：决定完成。 ---")
        return "__end__"

print("路由器函数已定义。")

## 阶段3：组装和运行工作流

### 步骤3.1：构建和可视化图

**我们要做什么：**
我们将创建 `StateGraph` 并添加我们的节点和边。关键部分是我们使用 `router_function` 创建的条件边，形成智能体的主要推理循环：`agent -> router -> tool -> agent`。

In [ ]:
graph_builder = StateGraph(AgentState)

# 添加节点
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("call_tool", tool_node)

# 设置入口点
graph_builder.set_entry_point("agent")

# 添加条件路由器
graph_builder.add_conditional_edges(
    "agent",
    router_function,
)

# 添加从工具节点回到智能体的边，完成循环
graph_builder.add_edge("call_tool", "agent")

# 编译图
tool_agent_app = graph_builder.compile()

print("工具使用智能体图编译成功！")

# 可视化图
try:
    from IPython.display import Image, display
    png_image = tool_agent_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"图可视化失败：{e}。请确保已安装pygraphviz。")

**输出讨论：**
编译的图已经准备好了。可视化清楚地显示了智能体的推理循环。流程从 `agent` 节点开始。然后条件边（由菱形表示）路由流程。如果需要工具，它会转到 `call_tool`，输出被反馈给 `agent` 进行综合。如果不需要工具，流程会转到 `__end__`。这种结构完美地实现了工具使用模式。

### 步骤3.2：端到端执行

**我们要做什么：**
让我们运行智能体，处理一个它不可能从训练数据中知道的问题，迫使它使用网络搜索工具。我们将流式传输中间步骤以观察其推理过程的展开。

In [ ]:
user_query = "最新的科技新闻有哪些？"
initial_input = {"messages": [("user", user_query)]}

console.print(f"[bold cyan]🚀 启动工具使用工作流，请求：[/bold cyan] '{user_query}'\n")

for chunk in tool_agent_app.stream(initial_input, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("\n[bold green]✅ 工具使用工作流完成！[/bold green]")

## 阶段4：评估

### 步骤4.1：分析执行轨迹

**我们要做什么：**
通过查看上一步的流式输出，我们可以追踪智能体的确切推理过程。输出显示了流经图状态的不同消息类型（`AIMessage` 包含 `tool_calls`，`ToolMessage` 包含结果）。

**输出讨论：**
执行轨迹清楚地展示了工具使用模式的实际应用：
1. 打印的第一条消息来自 `agent` 节点。它是一个包含 `tool_calls` 属性的 `AIMessage`，表明LLM正确决定使用 `web_search` 工具。
2. 下一条消息是一个 `ToolMessage`。这是 `tool_node` 在执行搜索并返回原始结果之后的输出。
3. 最后一条消息是另一个 `AIMessage`，但这次没有 `tool_calls`。这是智能体将 `ToolMessage` 中的信息综合成连贯的、给用户的最终答案。
这个轨迹确认了智能体的逻辑和图的路由工作得完美。

### 步骤4.2：使用LLM作为评判者进行评估

**我们要做什么：**
我们将创建一个"评判"LLM，为智能体的性能提供结构化的、定量的评估。评估标准将专门针对评估工具使用的质量。

In [ ]:
class ToolUseEvaluation(BaseModel):
    """用于评估智能体工具使用和最终答案的模式。"""
    tool_selection_score: int = Field(description="智能体是否为任务选择了正确工具的1-5分评分。")
    tool_input_score: int = Field(description="对工具输入格式良好性和相关性的1-5分评分。")
    synthesis_quality_score: int = Field(description="智能体如何将工具输出整合到最终答案中的1-5分评分。")
    justification: str = Field(description="评分的简要理由。")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# 为了评估，我们需要重建完整的对话轨迹
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.content or ''} {getattr(m, 'tool_calls', '')}" for m in final_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""您是AI智能体的专家评判员。根据智能体的工具使用情况，从1-5的范围内评估以下对话轨迹。提供简要的理由。
    
    对话轨迹：
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- 评估工具使用性能 ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())

**输出讨论：**
LLM作为评判者提供了对我们智能体性能的结构化和有理由的评估。`tool_selection_score`、`tool_input_score` 和 `synthesis_quality_score` 三个类别的高分确认我们的智能体不仅仅是使用工具，而是*有效*地使用工具。它正确识别了需要网络搜索，制定了相关的查询，并成功地将检索到的事实综合成一个有用且准确的最终答案。这种自动化评估给了我们对实现健壮性的信心。

## 结论

在这个notebook中，我们已经构建了一个基于**工具使用**架构的完整、功能性智能体。我们成功地为Minimax驱动的LLM配备了网络搜索工具，并使用LangGraph创建了一个强大的推理循环，允许智能体决定何时以及如何使用它。

端到端执行和后续评估展示了这种模式的巨大价值。通过将我们的智能体连接到实时的外部信息，我们从根本上克服了静态训练数据的限制。智能体不再只是一个推理器；它是一个研究者，能够提供基于事实、准确且及时的答案。这种架构是创建 virtually任何实用的、真实世界AI助手的基础构建块。